# SCC0250 - Computer Graphics

Fernando Gonçalves Campos - 12542352\
José Pedro Cioni do Carmo - 12623988\
Thiago Shimada - 12691032

**Controls**\
Mouse            Camera view\
WASD             Camera movement\
↑←↓→             Model movement same direction of the camera movement\
QE                  Model rotation in the same axis of the camera looking direction (anticlockwise/clockwise)\
+-                   Model scaling\
NM                 Decelerate/Accelerate time pass\
P                     Toggles Polygon mode\
ESC                 Closes the window


**Obs :** Docstrings were created using Gemini and modified in some parts afterwards

### Imports

In [550]:
import glfw
from OpenGL.GL import *
import glm

import numpy as np
import OpenGL.arrays.numpymodule as glnp

import random

from PIL import Image

### Create Window

In [551]:
class WindowDim:
    """
    Represents the dimensions and viewing limits of a virtual window.
    """
    width: int = 2
    height: int = 2
    left_limit: float = -1
    right_limit: float = 1
    bottom_limit: float = -1
    top_limit: float = 1

    @classmethod
    def update(cls, width: int, height: int) -> None:
        """
        Updates the window dimensions and recalculates the boundaries.
        """
        cls.width = width
        cls.height = height

        cls.left_limit = -width / 2
        cls.right_limit = width / 2

        cls.bottom_limit = -height / 2
        cls.top_limit = height / 2

In [552]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

WindowDim.update(1600, 800)

window = glfw.create_window(WindowDim.width, WindowDim.height, "Computer Graphics", None, None)
glfw.make_context_current(window)

### Shaders

In [553]:
N_LIGHT_SOURCES: int = 2

vertex_code = """
    in vec3 attr_position;
    in vec2 attr_texture_coord;
    in vec3 attr_normal;

    out vec2 vertex_texture_coord;
    out vec3 vertex_position;
    out vec3 vertex_normal;

    uniform mat4 model_matrix;
    uniform mat4 view_matrix;
    uniform mat4 projection_matrix;

    void main() {
        gl_Position = projection_matrix * view_matrix * model_matrix * vec4(attr_position, 1.0);
        vertex_texture_coord = attr_texture_coord;
        vertex_position = vec3(model_matrix * vec4(attr_position, 1.0));
        vertex_normal = mat3(transpose(inverse(model_matrix))) * attr_normal;
    };
"""

fragment_code = f"""
    #define N_LIGHT_SOURCES {N_LIGHT_SOURCES}

    uniform vec3 light_position[N_LIGHT_SOURCES];
    uniform vec3 light_color[N_LIGHT_SOURCES];
    uniform float decay_distance_unit[N_LIGHT_SOURCES];

    out vec4 out_color;

    in vec2 vertex_texture_coord;
    in vec3 vertex_position;
    in vec3 vertex_normal;

    uniform vec3 view_position;

    uniform vec3 ka;
    uniform vec3 kd;
    uniform vec3 ks;
    uniform float ns;

    uniform vec3 ambient_light;

    uniform sampler2D uniform_texture;

    const vec4 max_light_effect = vec4(1.1,1.1,1.1,1.0);

    void main() {{
        const vec3 ambient = ka * ambient_light;

        const vec3 norm = normalize(vertex_normal);
        const vec3 view_direction = normalize(view_position - vertex_position);
        vec3 diffuse = vec3(0,0,0);
        vec3 specular = vec3(0,0,0);

        for(int i = 0; i < N_LIGHT_SOURCES; ++i){{
            const float vertex_view_distance = length(vertex_position - view_position) / decay_distance_unit[i];
            const float light_vertex_distance = length(light_position[i] - vertex_position) / decay_distance_unit[i];

            const float vv_distance_square = vertex_view_distance * vertex_view_distance;
            const float lv_distance_square = light_vertex_distance * light_vertex_distance;

            const vec3 light_direction = normalize(light_position[i] - vertex_position);
            const float diff = max(dot(norm, light_direction), 0.0);
            const vec3 cur_diffuse = (kd * diff * light_color[i]) / lv_distance_square;
            diffuse += min(cur_diffuse, kd * light_color[i]);

            const vec3 reflection_direction = normalize(reflect(-light_direction, norm));
            const float spec = pow(max(dot(view_direction, reflection_direction), 0.0), ns * lv_distance_square);
            const vec3 cur_specular = (ks * spec * light_color[i]) / (lv_distance_square + vv_distance_square);
            specular += min(cur_specular, ks * light_color[i]);
        }}

        const vec4 resulting_light = vec4(ambient + diffuse + specular, 1.0);
        const vec4 texture_color = texture(uniform_texture, vertex_texture_coord);

        out_color = min(resulting_light, max_light_effect) * texture_color;
    }};
"""

#### Shaders Compilation

In [554]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

glEnable(GL_DEPTH_TEST)
glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
glEnable(GL_BLEND)
glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
glEnable(GL_LINE_SMOOTH)
glEnable(GL_TEXTURE_2D)

In [555]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

In [556]:
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

In [557]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

In [558]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

In [559]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')

# Make program the default program
glUseProgram(program)

### Object classes

#### Keyboard Classes

In [560]:
class KeyInputs:
    """
    This class stores the state (pressed or not) of various keys.
    """
    SPACE: int = 0
    P: int = 0
    W: int = 0
    A: int = 0
    S: int = 0
    D: int = 0
    Q: int = 0
    E: int = 0
    M: int = 0
    N: int = 0
    UP: int = 0
    LEFT: int = 0
    DOWN: int = 0
    RIGHT: int = 0
    MINUS: int = 0
    PLUS: int = 0
    ESC: int = 0

    @classmethod
    def get_inputs(cls, window) -> None:
        """
        Gets the state of all the keys stored in the class attributes.
        """
        cls.SPACE = glfw.get_key(window, glfw.KEY_SPACE)
        cls.P = glfw.get_key(window, glfw.KEY_P)
        cls.W = glfw.get_key(window, glfw.KEY_W)
        cls.A = glfw.get_key(window, glfw.KEY_A)
        cls.S = glfw.get_key(window, glfw.KEY_S)
        cls.D = glfw.get_key(window, glfw.KEY_D)
        cls.Q = glfw.get_key(window, glfw.KEY_Q)
        cls.E = glfw.get_key(window, glfw.KEY_E)
        cls.M = glfw.get_key(window, glfw.KEY_M)
        cls.N = glfw.get_key(window, glfw.KEY_N)
        cls.UP = glfw.get_key(window, glfw.KEY_UP)
        cls.LEFT = glfw.get_key(window, glfw.KEY_LEFT)
        cls.DOWN = glfw.get_key(window, glfw.KEY_DOWN)
        cls.RIGHT = glfw.get_key(window, glfw.KEY_RIGHT)
        cls.MINUS = glfw.get_key(window, glfw.KEY_MINUS)
        cls.PLUS = glfw.get_key(window, glfw.KEY_EQUAL)
        cls.ESC = glfw.get_key(window, glfw.KEY_ESCAPE)

#### Projection Classes

In [561]:
class Projection:
    """
    This class handles the creation and configuration of a perspective projection matrix.
    """
    fovy: float = glm.pi() / 4
    near: float = 0.01
    far: float = 1

    @classmethod
    def get_projection_matrix(cls) -> glm.mat4:
        """
        Returns the perspective projection matrix based on the current configuration.
        """
        return glm.perspective(cls.fovy, WindowDim.width / WindowDim.height, cls.near, cls.far)

    @classmethod
    def set_fovy(cls, fovy: float) -> None:
        """
        Sets the field-of-view angle of the projection matrix.
        """
        cls.fovy = fovy

    @classmethod
    def set_near(cls, near: float) -> None:
        """
        Sets the distance to the near clipping plane of the projection matrix.
        """
        cls.near = near

    @classmethod
    def set_far(cls, far: float) -> None:
        """
        Sets the distance to the far clipping plane of the projection matrix.
        """
        cls.far = far

Projection.set_fovy(glm.pi() / 4)
Projection.set_near(0.1)
Projection.set_far(1000)

#### Camera Classes

In [562]:
class Camera:
    """
    Handles the camera controls.
    """
    speed: float = 1
    position: glm.vec3 = glm.vec3(-12.0, 4.0, -20.0)
    front: glm.vec3 = glm.vec3(0.0, 0.0, 1.0)
    up: glm.vec3 = glm.vec3(0.0, 1.0, 0.0)

    sensitivity: bool = 1
    yaw: float = - glm.pi() / 2
    pitch: float = 0.0

    first_mouse_position: bool = True
    last_cursor_position_x: float = 0
    last_cursor_position_y: float = 0

    @classmethod
    def handle_movement(cls, delta_time: float) -> None:
        """
        Updates the camera position based on keyboard input (W, A, S, D) and movement speed.

        Args:
            delta_time: Time elapsed since the last frame (in seconds).
        """
        movement: float = cls.speed * delta_time
        radius = 138

        # Updates the camera position
        if KeyInputs.W == glfw.PRESS or KeyInputs.W == glfw.REPEAT:
            cls.position += cls.front * movement
        if KeyInputs.S == glfw.PRESS or KeyInputs.S == glfw.REPEAT:
            cls.position -= cls.front * movement
        if KeyInputs.A == glfw.PRESS or KeyInputs.A == glfw.REPEAT:
            cls.position -= glm.normalize(glm.cross(cls.front, cls.up)) * movement
        if KeyInputs.D == glfw.PRESS or KeyInputs.D == glfw.REPEAT:
            cls.position += glm.normalize(glm.cross(cls.front, cls.up)) * movement

        # Handles floor collision
        if cls.position.y < 0.8:
            cls.position.y = 0.8

        # Handles skybox collision
        if glm.length(cls.position) >= radius:
            cls.position = glm.normalize(cls.position) * (radius - 0.001)

        loc_view_position = glGetUniformLocation(program, "view_position")
        glUniform3f(loc_view_position, cls.position.x, cls.position.y, cls.position.z)

    @classmethod
    def mouse_movement(cls, window, cursor_position_x: float, cursor_position_y: float) -> None:
        """
        Updates the camera direction based on mouse movement and sensitivity.
        """

        # Saves the first mouse position as its previous position (since in this situation there wasn't any previous position)
        if cls.first_mouse_position:
            cls.last_cursor_position_x = cursor_position_x
            cls.last_cursor_position_y = cursor_position_y
            cls.first_mouse_position = False

        # Calculates how much the mouse moved in relation to its last position
        offset_x = (cursor_position_x - cls.last_cursor_position_x) * cls.sensitivity
        offset_y = (cls.last_cursor_position_y - cursor_position_y) * cls.sensitivity

        # Saves the current mouse position as the "previous" position
        cls.last_cursor_position_x = cursor_position_x
        cls.last_cursor_position_y = cursor_position_y

        # Updates the camera angles
        cls.yaw += offset_x
        cls.pitch += offset_y

        # Prevents the camera from looping vertically
        if cls.pitch >  glm.pi()/2 - 0.005 : cls.pitch =  glm.pi()/2 - 0.005
        if cls.pitch < -glm.pi()/2 + 0.005: cls.pitch = -glm.pi()/2 + 0.005

        # Updates the camera front based on its angle
        cls.front.x = glm.cos(cls.yaw) * glm.cos(cls.pitch)
        cls.front.y = glm.sin(cls.pitch)
        cls.front.z = glm.sin(cls.yaw) * glm.cos(cls.pitch)
        cls.front = glm.normalize(cls.front)

    @classmethod
    def get_view_matrix(cls) -> glm.mat4:
        """
        Calculates the view matrix representing the camera's current position and orientation.
        """
        return glm.lookAt(cls.position, cls.position + cls.front, cls.up)

    @classmethod
    def set_speed(cls, speed: float) -> None:
        """
        Sets the movement speed of the camera.
        """
        cls.speed = speed

    @classmethod
    def set_sensitivity(cls, sensitivity: float) -> None:
        """
        Sets the sensitivity for camera rotation based on mouse movement.
        """
        cls.sensitivity = sensitivity

# Camera values
Camera.set_sensitivity(0.3 * glm.pi() / 180)
Camera.set_speed(20)

# Configures the mouse callback
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)
glfw.set_cursor_pos_callback(window, Camera.mouse_movement)

#### Textures

In [563]:
class Material:
    def __init__(
            self,
            ambient_color: glm.vec3,
            diffuse_color: glm.vec3,
            specular_color: glm.vec3,
            specular_highlights: float,
            optical_density: float,
            dissolve: float,
            ilumination_model: int,
            texture_filename: str
        ):
        self.ambient_color: glm.vec3= ambient_color
        self.diffuse_color: glm.vec3 = diffuse_color
        self.specular_color: glm.vec3 = specular_color
        self.specular_highlights: float = specular_highlights
        self.optical_density: float = optical_density
        self.dissolve: float = dissolve
        self.ilumination_model: int = ilumination_model
        self._load_texture_from_file(texture_filename)

    def __exit__(self, exc_type, exc_value, exc_traceback):
        """
        Releases the texture associated with the material
        """
        glDeleteTextures([self.texture_id])

    def bind(self) -> None:
        """
        Passes the information from the material to the shaders
        """
        glBindTexture(GL_TEXTURE_2D, self.texture_id)

        loc_ka = glGetUniformLocation(program, "ka")
        glUniform3f(loc_ka, self.ambient_color.x, self.ambient_color.y, self.ambient_color.z)

        loc_kd = glGetUniformLocation(program, "kd")
        glUniform3f(loc_kd, self.diffuse_color.x, self.diffuse_color.y, self.diffuse_color.z)

        loc_ks = glGetUniformLocation(program, "ks")
        glUniform3f(loc_ks, self.specular_color.x, self.specular_color.y, self.specular_color.z)

        loc_ns = glGetUniformLocation(program, "ns")
        glUniform1f(loc_ns, self.specular_highlights)

    def _load_texture_from_file(self, filename: str) -> None:
        """
        Loads a texture from a file.

        Args:
            filename: The path to the image file to load.
        """
        self.texture_id: int = glGenTextures(1)

        glBindTexture(GL_TEXTURE_2D, self.texture_id)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)

        # Reads the image
        img: Image.Image = Image.open(filename)
        img_width = img.size[0]
        img_height = img.size[1]
        image_data = img.convert("RGBA").tobytes("raw", "RGBA",0,-1)

        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, img_width, img_height, 0, GL_RGBA, GL_UNSIGNED_BYTE, image_data)

#### Model Classes

##### Movement

In [564]:
from abc import ABC, abstractmethod

class Movement(ABC):
    """
    Abstract base class representing any transformation applied to a 3D model during runtime.

    This class defines the interface for updating the transformation values based on
    elapsed time (delta_time) and retrieving the resulting transformation matrix.
    """

    @abstractmethod
    def update_values(self, delta_time: float) -> None:
        """
        Updates the internal state of the movement based on the elapsed time (delta_time).

        Args:
            delta_time: Time elapsed since the last update in seconds (float).
        """
        pass

    @abstractmethod
    def get_transformation_matrix(self) -> glm.mat4:
        """
        Returns the current transformation matrix representing the applied movement.

        Returns:
            A transformation matrix as a glm.mat4 object.
        """
        pass

class ControledScale(Movement):
    """
    Movement that scales a model over time based on user input (-/+).

    Args:
      time_to_double: Time it takes for the scale to double (float).
      starting_scale: Initial scale of the model (default: 1.0).
    """
    def __init__(self, time_to_double: float, starting_scale: float = 1):
        self.scale: float = starting_scale
        self.growth_speed: float = 1/time_to_double

    def update_values(self, delta_time: float) -> None:
        if KeyInputs.PLUS == glfw.PRESS or KeyInputs.PLUS == glfw.REPEAT:
            self.scale *= glm.exp2(self.growth_speed * delta_time)
        if KeyInputs.MINUS == glfw.PRESS or KeyInputs.MINUS == glfw.REPEAT:
            self.scale *= glm.exp2(-self.growth_speed * delta_time)

    def get_transformation_matrix(self) -> glm.mat4:
        return glm.scale(glm.vec3(self.scale))

class ControledTranslationBasedOnCamera(Movement):
    """
    Movement that translates a model based on user input and the camera's orientation.

    Args:
      speed (float, optional): The movement speed in units per second. Defaults to 1.0.
      starting_position (glm.vec3, optional): The initial position of the model. Defaults to glm.vec3(0.0, 0.0, 0.0).
    """
    def __init__(self, speed: float = 1.0, starting_position: glm.vec3 = glm.vec3(0.0,0.0,0.0)):
        self.speed = speed
        self.position: glm.vec3 = starting_position

    def update_values(self, delta_time: float) -> None:
        movement: float = self.speed * delta_time

        if KeyInputs.UP == glfw.PRESS or KeyInputs.UP == glfw.REPEAT:
            self.position += Camera.front * movement
        if KeyInputs.DOWN == glfw.PRESS or KeyInputs.DOWN == glfw.REPEAT:
            self.position -= Camera.front * movement
        if KeyInputs.LEFT == glfw.PRESS or KeyInputs.LEFT == glfw.REPEAT:
            self.position -= glm.normalize(glm.cross(Camera.front, Camera.up)) * movement
        if KeyInputs.RIGHT == glfw.PRESS or KeyInputs.RIGHT == glfw.REPEAT:
            self.position += glm.normalize(glm.cross(Camera.front, Camera.up)) * movement

    def get_transformation_matrix(self) -> glm.mat4:
        return glm.translate(self.position)

class ControledRotationBasedOnCamera(Movement):
    """
    Movement that rotates a model based on user input (Q and E keys)
    relative to the camera's front vector.

    Args:
        angular_velocity: The speed of rotation in radians per second (float).
    """
    def __init__(self, angular_velocity: float):
        self.angular_velocity: float = angular_velocity
        self.rotation: glm.mat4 = glm.mat4(1.0)

    def update_values(self, delta_time: float) -> None:
        if KeyInputs.Q == glfw.PRESS or KeyInputs.Q == glfw.REPEAT:
            self.rotation = glm.rotate(-self.angular_velocity * delta_time, Camera.front) * self.rotation
        if KeyInputs.E == glfw.PRESS or KeyInputs.E == glfw.REPEAT:
            self.rotation = glm.rotate(self.angular_velocity * delta_time, Camera.front) * self.rotation

    def get_transformation_matrix(self) -> glm.mat4:
        return self.rotation

class CyclicRotation(Movement):
    """
    Represents a cyclic rotation movement applied to a 3D model.

    Args:
        duration (float): The duration (in seconds) for a complete rotation.
        starting_angle (float, optional): The starting angle in radians (defaults to 0.0).
        clockwise (bool, optional): Determines the direction of rotation (defaults to True).
        axis (glm.vec3, optional): The axis around which the rotation occurs (defaults to glm.vec3(0.0, 1.0, 0.0)).
    """
    def __init__(self, duration: float, starting_angle: float = 0.0, clockwise: bool = True, axis: glm.vec3 = glm.vec3(0.0, 1.0, 0.0)):
        self.full_rotation: float = 2 * glm.pi()

        self.angle: float = starting_angle
        self.angular_velocity: float = self.full_rotation / duration

        if not clockwise:
            self.angular_velocity *= -1

        self.axis: glm.vec3 = axis

    def update_values(self, delta_time: float) -> None:
        self.angle += self.angular_velocity * delta_time
        self.angle %= self.full_rotation

    def get_transformation_matrix(self) -> glm.mat4:
        return glm.rotate(self.angle, self.axis)

class CyclicOrbit(Movement):
    """
    Represents a cyclic orbit around a specified axis.

    Args:
        duration (float): Time taken to complete a full rotation in seconds.
        radius (float): Distance from the center of the orbit to the object.
        starting_angle (float, optional): The starting angle of the orbit in radians. Defaults to 0.0.
        clockwise (bool, optional): Determines the direction of the orbit. True for clockwise, False for counter-clockwise. Defaults to True.
        axis (glm.vec3, optional): The axis of rotation. Defaults to glm.vec3(0.0, 1.0, 0.0) (y-axis).
    """
    def __init__(self, duration: float, radius: float, starting_angle: float = 0.0, clockwise: bool = True, axis = glm.vec3(0.0, 1.0, 0.0)):
        self.full_rotation: float = 2 * glm.pi()

        self.radius: float = radius

        self.angle: float = starting_angle
        self.angular_velocity: float = self.full_rotation / duration

        if not clockwise:
            self.angular_velocity *= -1

        self.axis: glm.vec3 = axis

    def update_values(self, delta_time: float) -> None:
        self.angle += self.angular_velocity * delta_time
        self.angle %= self.full_rotation

    def get_transformation_matrix(self) -> glm.mat4:
        rotation: glm.mat4 = glm.rotate(self.angle, self.axis)

        # Calculating the perpendicular direction from the rotation axis to translate the model the specified radius
        # x(ay + bz) + y(cx + dz) + z(ex + fy) == 0
        # a = -c
        # b = -e
        # d = -f
        perpendicular_direction: glm.vec3 = glm.vec3(self.axis.y + self.axis.z, self.axis.z - self.axis.x, -self.axis.x - self.axis.y)
        return glm.translate(rotation, glm.normalize(perpendicular_direction) * self.radius)

##### Model Types

Main Model

In [565]:
class Model:
    """
    Represents a 3D model loaded from a Wavefront OBJ file.

    This class encapsulates the data and functionality for rendering a 3D model.
    It supports loading the model from a file, adding and applying transformations,
    and rendering the model with a texture.

    Attributes:
        filename: Path to the Wavefront OBJ file.
        texture_filename: Path to the texture image file.
        movements: List of Movement objects that define animations for the model.
        texture_id: ID of the loaded texture.
        fixed_starting_transformation: Transformation matrix applied before animations.
        fixed_ending_transformation: Transformation matrix applied after animations.
        indices: Array of indices referencing vertices for triangle rendering.
    """
    def __init__(self, filename: str):
        self.movements: list[Movement] = []

        self.fixed_starting_transformation: glm.mat4 = glm.mat4(1)
        self.fixed_ending_transformation: glm.mat4 = glm.mat4(1)

        self.materials_hash: dict[str, int] = {}
        self.materials: list[Material] = []
        self.material_indices: list[int] = []
        self.indices: list[np.ndarray] = []

        self.vertex_array: int = 0
        self.vertex_buffer: int = 0
        self.index_buffers: list[int] = []

        self.model_matrix: glm.mat4 = glm.mat4(1)

        self._load_model_from_file(filename)

    def __exit__(self, exc_type, exc_value, exc_traceback):
        """
        Releases resources associated with the model, including OpenGL buffers, textures, and vertex arrays.
        """
        glDeleteBuffers(1, [self.vertex_buffer])
        glDeleteBuffers(len(self.index_buffers), self.index_buffers)
        glDeleteVertexArrays(1, [self.vertex_array])

    def add_starting_transformation(self, transformation: glm.mat4) -> None:
        """
        Adds a transformation matrix to be applied before any animations are applied to the model.

        This transformation is pre-multiplied with the existing starting transformation,
        resulting in a combined transformation applied at the beginning of model transformation matrix.

        Args:
            transformation: A 4x4 transformation matrix in glm.mat4 format.
        """
        self.fixed_starting_transformation = transformation * self.fixed_starting_transformation

    def add_ending_transformation(self, transformation: glm.mat4) -> None:
        """
        Adds a transformation matrix to be applied after any animations are applied to the model.

        This transformation is pre-multiplied with the existing ending transformation,\
        resulting in a combined transformation applied at the end of the model transformation matrix.

        Args:
            transformation: A 4x4 transformation matrix in glm.mat4 format.
        """
        self.fixed_ending_transformation = transformation * self.fixed_ending_transformation

    def add_movement(self, movement: Movement) -> None:
        """
        Adds a Movement object to the model's animation list.

        This function allows you to define animations for the model. The provided
        Movement object encapsulates the animation logic and updates the model's
        transformation based on time or other factors.

        Args:
            movement: A Movement object representing the animation to be added.
        """
        self.movements.append(movement)

    def render(self, extra_transformation: glm.mat4 = glm.mat4(1)) -> None:
        """
        Renders the model on the screen.

        Args:
            inherited_matrix: A 4x4 matrix representing the transformations of the model's group.
        """
        glBindVertexArray(self.vertex_array)

        loc = glGetUniformLocation(program, 'model_matrix')

        model_matrix: glm.mat4 = extra_transformation * self.model_matrix
        glUniformMatrix4fv(loc, 1, GL_FALSE, model_matrix.to_bytes())

        # Renders each part of the model that uses a different material separately
        for i in range(len(self.index_buffers)):
            glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, self.index_buffers[i])
            self.materials[self.material_indices[i]].bind()
            glDrawElements(GL_TRIANGLES, self.indices[i].shape[0], GL_UNSIGNED_INT, None)

    def handle_movement(self, delta_time: float, inherited_transformation: glm.mat4 = glm.mat4(1)) -> None:
        """
        Updates the animations of the model and its submodels (if it's a CompoundModel).

        Args:
            delta_time: Time elapsed since the last frame (in seconds).
        """
        for movement in self.movements:
            movement.update_values(delta_time)

        self._update_model_matrix(inherited_transformation)

    def _update_model_matrix(self, inherited_transformation: glm.mat4 = glm.mat4(1)) -> None:
        """
        Calculates the model matrix for rendering.

        This function combines the following transformations to create the final
        model matrix:

        1. Fixed starting transformation (applied before animations).
        2. Transformations from all attached Movement objects (applied in order).
        3. Fixed ending transformation (applied after animations).

        Returns:
            The model matrix as a glm.mat4 object.
        """
        model_matrix: glm.mat4 = self.fixed_starting_transformation
        for movement in self.movements:
            model_matrix = movement.get_transformation_matrix() * model_matrix
        model_matrix = self.fixed_ending_transformation * model_matrix

        self.model_matrix = inherited_transformation * model_matrix

    def _update_ka(self, new_ka: glm.vec3) -> None:
        for material in self.materials:
            material.ambient_color = new_ka

    def _load_model_from_file(self, filename: str) -> None:
        """
        Loads a Wavefront OBJ file and parses its data into to OpenGL buffers. Also saves all information needed for rendering.

        Args:
            filename: Path to the Wavefront OBJ file.
        """
        vertex_coords: list[list[int]] = []
        texture_coords: list[list[int]] = []
        normals: list[list[int]] = []

        vertices_hash: dict[str, int] = {}
        vertices: list[int] = []
        cur_indices: list[int] = []

        next_indice: int = 0

        material: int = -1

        # Opens the .obj file
        for line in open(filename, "r"): # Read all lines of the file
            if line.startswith('#'): continue # Skips comments

            values = line.split() # Split the line in spaces
            if not values: continue

            # Creates new materials
            if values[0] == 'mtllib':
                self._load_materials(" ".join(values[1:]))

            # Store the vertex position
            elif values[0] == 'v':
                vertex_coords.append([float(value) for value in values[1:4]])

            # Stores the texture coordinates
            elif values[0] == 'vt':
                texture_coords.append([float(value) for value in values[1:3]])

            # Stores the vertex normal
            elif values[0] == 'vn':
                normals.append([float(value) for value in values[1:4]])

            # Stores the material used
            elif values[0] in ('usemtl', 'usemat'):
                # Saves the indices used with the previous material
                if material != -1 and len(cur_indices) != 0:
                    self.indices.append(np.array(cur_indices, dtype=np.uint32))
                    cur_indices = []
                    self.material_indices.append(material)

                material = self.materials_hash[values[1]] if values[1] in self.materials_hash else -1

            # Stores the face
            elif values[0] == 'f':
                face_indices = []

                for value in values[1:]:
                    # Checks if the same vertex was already stored
                    if value in vertices_hash:
                        face_indices.append(vertices_hash[value])
                    else:
                        # Saves the vertex ID
                        vertices_hash[value] = next_indice
                        face_indices.append(next_indice)
                        next_indice += 1

                        # Stores the vertex data
                        vertex_attrs = value.split('/')
                        cur_vertex: list[int] = [int(attr_id)-1 for attr_id in vertex_attrs]
                        while len(cur_vertex) < 3:
                            cur_vertex.append(0)

                        vertices.append(cur_vertex)

                # Converts the face into triangles
                for i in range(1, len(face_indices)-1):
                    cur_indices.append(face_indices[0])
                    cur_indices.append(face_indices[i])
                    cur_indices.append(face_indices[i+1])

        # Store the remainder of the indices
        if material != -1 and len(cur_indices) != 0:
            self.indices.append(np.array(cur_indices, dtype=np.uint32))
            cur_indices = []
            self.material_indices.append(material)

        # Converts the lists to numpy arrays
        vertices_values: np.ndarray = np.array(vertices)
        vertices_coords: np.ndarray = np.array(vertex_coords)
        texture_coords: np.ndarray = np.array(texture_coords)
        normals: np.ndarray = np.array(normals)

        # Stores the vertex attributes in the same structured array
        model_vertices: np.ndarray = np.zeros(vertices_values.shape[0], [("vertex_coords", np.float32, 3), ("texture_coords", np.float32, 2), ("normal", np.float32, 3)])
        model_vertices["vertex_coords"] = vertices_coords[vertices_values[:,0]]
        model_vertices["texture_coords"] = texture_coords[vertices_values[:,1]]
        model_vertices["normal"] = normals[vertices_values[:,2]]

        # Create the OpenGL buffers
        self._create_buffers(model_vertices)

    def _load_materials(self, filename: str):
        """
        Loads a Wavefront MTL file and parses its data materials. Saves the materials loaded into a hash table.

        Args:
            filename: Path to the Wavefront MTL file.
        """
        material_name: str = ""
        ambient_color: glm.vec3 = glm.vec3(1.0)
        diffuse_color: glm.vec3 = glm.vec3(1.0)
        specular_color: glm.vec3 = glm.vec3(1.0)
        specular_highlights: float = 32
        optical_density: float = 0
        dissolve: float = 0
        ilumination_model: int = 0
        texture_filename: str = ""

        # Opens the .obj file
        for line in open(filename, "r"): # Read all lines of the file
            if line.startswith('#'): continue # Skips comments

            values = line.split() # Split the line in spaces
            if not values: continue

            # Initializes a new material and saves the previous one
            if values[0] == 'newmtl':
                if material_name != '': # Prevents from saving unamed materials
                    self.materials_hash[material_name] = len(self.materials)
                    self.materials.append(
                        Material(
                            ambient_color,
                            diffuse_color,
                            specular_color,
                            specular_highlights,
                            optical_density,
                            dissolve,
                            ilumination_model,
                            texture_filename
                        )
                    )

                # Initializes the values of the new material
                material_name = " ".join(values[1:])
                ambient_color = glm.vec3(1.0)
                diffuse_color = glm.vec3(1.0)
                specular_color = glm.vec3(1.0)
                specular_highlights = 32
                optical_density = 0
                dissolve = 0
                ilumination_model = 0
                texture_filename = ""

            # Stores the ambient color
            elif values[0] == "Ka":
                ambient_color.x = float(values[1])
                ambient_color.y = float(values[2])
                ambient_color.z = float(values[3])

            # Stores the diffuse color
            elif values[0] == "Kd":
                diffuse_color.x = float(values[1])
                diffuse_color.y = float(values[2])
                diffuse_color.z = float(values[3])

            # Stores the specular color
            elif values[0] == "Ks":
                specular_color.x = float(values[1])
                specular_color.y = float(values[2])
                specular_color.z = float(values[3])

            # Stores the specular highlights
            elif values[0] == "Ns":
                specular_highlights = float(values[1])

            # Stores de optical density (refraction)
            elif values[0] == "Ni":
                optical_density = float(values[1])

            # Stores the dissolve value (opacity)
            elif values[0] == "d":
                dissolve = float(values[1])

            # Stores the illumination model that should be used
            elif values[0] == "illum":
                ilumination_model = int(values[1])

            # Stores the diffuse texture
            elif values[0] == "map_Kd":
                texture_filename = " ".join(values[1:])

        # Saves the last material read
        if material_name != '':
            self.materials_hash[material_name] = len(self.materials)
            self.materials.append(
                Material(
                    ambient_color,
                    diffuse_color,
                    specular_color,
                    specular_highlights,
                    optical_density,
                    dissolve,
                    ilumination_model,
                    texture_filename
                )
            )

    def _create_buffers(self, model_vertices: np.ndarray) -> None:
        """
        Creates OpenGL vertex and element array buffers for rendering the model.

        This function takes the model data in a NumPy array format and uploads it
        to GPU memory using vertex and element array buffers. It also configures
        vertex attribute pointer to specify the location and data format of the array.

        Args:
            model_vertices: A NumPy array containing interleaved vertex data with attributes like positions and texture coordinates.
        """
        # Saves the information of which vertices should be used for each shader attribute
        self.vertex_array = glGenVertexArrays(1)
        glBindVertexArray(self.vertex_array)

        # Saves the vertices
        self.vertex_buffer = glGenBuffers(1)
        glBindBuffer(GL_ARRAY_BUFFER, self.vertex_buffer)
        glBufferData(GL_ARRAY_BUFFER, model_vertices.nbytes, model_vertices, GL_DYNAMIC_DRAW)

        stride = model_vertices.strides[0]

        # Vertices positions
        offset = ctypes.c_void_p(0)
        loc_positions = glGetAttribLocation(program, "attr_position")
        glEnableVertexAttribArray(loc_positions)
        glVertexAttribPointer(loc_positions, 3, GL_FLOAT, False, stride, offset)

        # Texture coordinates
        offset = ctypes.c_void_p(12)
        loc_texture_coords = glGetAttribLocation(program, "attr_texture_coord")
        glEnableVertexAttribArray(loc_texture_coords)
        glVertexAttribPointer(loc_texture_coords, 2, GL_FLOAT, False, stride, offset)

        # Normals
        offset = ctypes.c_void_p(20)
        loc_normal = glGetAttribLocation(program, "attr_normal")
        glEnableVertexAttribArray(loc_normal)
        glVertexAttribPointer(loc_normal, 3, GL_FLOAT, False, stride, offset)

        # Saves the indices of the vertices used for drawing
        for cur_indices in self.indices:
            index_buffer = glGenBuffers(1)
            glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, index_buffer)
            glBufferData(GL_ELEMENT_ARRAY_BUFFER, cur_indices.nbytes, cur_indices, GL_STATIC_DRAW)
            self.index_buffers.append(index_buffer)

Compound Model

In [566]:
class CompoundModel(Model):
    """
    Represents a compound 3D model composed of multiple submodels.

    This class inherits from the Model class and extends its functionality to handle
    a collection of submodels. It allows for rendering a complex model composed
    of multiple individual models.

    Attributes:
        movements: List of Movement objects that define animations for the entire compound model.
        submodels: List of Model objects representing the individual models within the compound.
        fixed_starting_transformation: Transformation matrix applied before animations.
        fixed_ending_transformation: Transformation matrix applied after animations.
    """
    def __init__(self):
        self.movements: list[Movement] = []
        self.submodels: list[Model] = []

        self.fixed_starting_transformation: glm.mat4 = glm.mat4(1)
        self.fixed_ending_transformation: glm.mat4 = glm.mat4(1)

        self.model_matrix: glm.mat4 = glm.mat4(1)

    def __exit__(self, exc_type, exc_value, exc_traceback):
        pass

    def add_model(self, model: Model) -> None:
        """
        Adds a submodel to the compound model.

        Args:
            model: The Model object representing the submodel to be added.
        """
        self.submodels.append(model)

    def handle_movement(self, delta_time: float, inherited_transformation: glm.mat4 = glm.mat4(1)) -> None:
        # Update own movements
        for movement in self.movements:
            movement.update_values(delta_time)

        self._update_model_matrix(inherited_transformation)

        # Update movements from the submodels
        for submodel in self.submodels:
            submodel.handle_movement(delta_time, inherited_transformation = self.model_matrix)

    def render(self, extra_transformation: glm.mat4 = glm.mat4(1)) -> None:
        # Render the submodels
        for submodel in self.submodels:
            submodel.render(extra_transformation)

    def _update_ka(self, new_ka: glm.vec3) -> None:
        for submodel in self.submodels:
            submodel._update_ka(new_ka)

Repeating Model

In [567]:
class RepeatingModel(Model):
    """
    Used for rendering the same model in different positions by specifying the offsets from the original model
    """
    def __init__(self, base_model: Model):
        self.base_model: Model = base_model
        self.offsets: list[glm.vec3] = [glm.vec3(0.0)]

    def add_offset(self, offset: glm.vec3) -> None:
        self.offsets.append(offset)

    def handle_movement(self, delta_time: float, inherited_transformation: glm.mat4 = glm.mat4(1)) -> None:
        self.base_model.handle_movement(delta_time, inherited_transformation)

    def render(self, extra_transformation: glm.mat4 = glm.mat4(1.0)) -> None:
        for offset in self.offsets:
            self.base_model.render(extra_transformation * glm.translate(offset))

    def _update_ka(self, new_ka: glm.vec3) -> None:
        self.base_model._update_ka(new_ka)

    @property
    def model_matrix(self) -> glm.mat4:
        return self.base_model.model_matrix

#### Light Source Classes

In [568]:
class LightSource:
    def __init__(
            self,
            light_id: int,
            reference_model: Model,
            light_color: glm.vec3,
            decay_distance_unit: float,
            position_offset: glm.vec3 = glm.vec3(0),
            time_to_double_decay_distance: None | float = None
        ):
        self.light_id: int = light_id
        self.reference_model: Model = reference_model
        self.light_color: glm.vec3 = light_color
        self.decay_distance_unit: float = decay_distance_unit
        self.position_offset: glm.vec3 = position_offset

        self.can_grow: bool = (time_to_double_decay_distance != None)
        self.growth_speed: float = 1/time_to_double_decay_distance if self.can_grow else 0

    def update(self, delta_time: float = 0) -> None:
        homogeneous_coords: glm.vec4 = glm.vec4(self.position_offset.x, self.position_offset.y, self.position_offset.z, 1)
        position: glm.vec3 = glm.vec3(self.reference_model.model_matrix * homogeneous_coords)

        if self.can_grow:
            if KeyInputs.PLUS == glfw.PRESS or KeyInputs.PLUS == glfw.REPEAT:
                self.decay_distance_unit *= glm.exp2(self.growth_speed * delta_time)
            if KeyInputs.MINUS == glfw.PRESS or KeyInputs.MINUS == glfw.REPEAT:
                self.decay_distance_unit *= glm.exp2(-self.growth_speed * delta_time)

        loc_light_position = glGetUniformLocation(program, f"light_position[{self.light_id}]")
        glUniform3f(loc_light_position, position.x, position.y, position.z)

        loc_decay_distance_unit = glGetUniformLocation(program, f"decay_distance_unit[{self.light_id}]")
        glUniform1f(loc_decay_distance_unit, self.decay_distance_unit)

        loc_light_color = glGetUniformLocation(program, f"light_color[{self.light_id}]")
        glUniform3f(loc_light_color, self.light_color.x, self.light_color.y, self.light_color.z)

class Illumination:
    ambient_light: glm.vec3 = glm.vec3(0.1)
    light_sources: list[LightSource] = []
    next_light_id: int = 0

    @classmethod
    def add_light_source(
        cls,
        reference_model: Model,
        light_color: glm.vec3,
        decay_distance_unit: float,
        position_offset: glm.vec3 = glm.vec3(0),
        time_to_double_decay_distance: None | float = None
    )-> None:
        reference_model._update_ka(glm.vec3(1/cls.ambient_light))
        cls.light_sources.append(
            LightSource(
                cls.next_light_id,
                reference_model,
                light_color,
                decay_distance_unit,
                position_offset=position_offset,
                time_to_double_decay_distance=time_to_double_decay_distance
            )
        )
        cls.next_light_id = (cls.next_light_id + 1) % N_LIGHT_SOURCES

    @classmethod
    def update(cls, delta_time: float = 0) -> None:
        for light_source in cls.light_sources:
            light_source.update(delta_time)

        loc_ambient_light = glGetUniformLocation(program, "ambient_light")
        glUniform3f(loc_ambient_light, cls.ambient_light.x, cls.ambient_light.y, cls.ambient_light.z)

    @classmethod
    def set_ambient_light(cls, ambient_light: glm.vec3) -> None:
        cls.ambient_light = ambient_light

### Create Models

In [569]:
models: list[Model] = []

##### Outside ground and skybox

In [570]:
def create_model_terreno() -> Model:
    model: Model = Model('objects/terreno/terreno2.obj')

    # Initial transformations
    model.add_starting_transformation(glm.scale(glm.vec3(200.0)))

    return model

def create_model_stone_path() -> Model:
    model: Model = Model("objects/stone path/stone path.obj")

    # Initial transformations
    model.add_starting_transformation(glm.scale(glm.vec3(31.0 * 1.8,1.0,1.8)))
    model.add_starting_transformation(glm.rotate(glm.pi()/2, glm.vec3(0,1,0)))

    # Final transformations
    model.add_ending_transformation(glm.translate(glm.vec3(-0.3, 0.05, 92.0)))

    return model

def create_model_sky() -> Model:
    model: Model = Model('objects/sky/sky.obj')

    # Initial transformations
    model.add_starting_transformation(glm.scale(glm.vec3(3.0)))

    return model

models.append(create_model_terreno())
models.append(create_model_stone_path())
models.append(create_model_sky())

##### Solar system

In [571]:
DAY_DURATION_SS: int = 0.025
DISTANCE_UNIT_SS: int = 4

def create_sun() -> Model:
    model: Model = Model("objects/sun/Sun.obj")

    # Initial transformations
    model.add_starting_transformation(glm.scale(glm.vec3(1.5)))

    # Movements
    model.add_movement(CyclicRotation(59 * DAY_DURATION_SS))

    Illumination.add_light_source(model, glm.vec3(0.9,0.9,0.9), 35, time_to_double_decay_distance=2)

    return model

def create_mercury() -> Model:
    model: Model = Model("objects/mercury/Mercury 1k.obj")

    # Initial transformations
    model.add_starting_transformation(glm.scale(glm.vec3(1.5)))

    # Movements
    model.add_movement(CyclicRotation(59 * DAY_DURATION_SS))
    model.add_movement(CyclicOrbit(88 * DAY_DURATION_SS, 0.39 * DISTANCE_UNIT_SS + 1.5))

    return model

def create_venus() -> Model:
    model: Model = Model("objects/venus/Venus_1K.obj")

    # Initial transformations
    model.add_starting_transformation(glm.scale(glm.vec3(0.8)))

    # Movements
    model.add_movement(CyclicRotation(243 * DAY_DURATION_SS))
    model.add_movement(CyclicOrbit(225 * DAY_DURATION_SS, 0.72 * DISTANCE_UNIT_SS + 1.5))

    return model

def create_earth() -> Model:
    model: Model = Model("objects/earth/Earth 2K.obj")

    # Initial transformations
    model.add_starting_transformation(glm.scale(glm.vec3(0.15)))

    # Movements
    model.add_movement(CyclicRotation(40 * DAY_DURATION_SS))

    return model

def create_moon() -> Model:
    model: Model = Model("objects/moon/Moon 2K.obj")

    # Initial transformations
    model.add_starting_transformation(glm.scale(glm.vec3(0.1)))

    # Movements
    model.add_movement(CyclicRotation(40 * DAY_DURATION_SS))
    model.add_movement(CyclicOrbit(16 * DAY_DURATION_SS, 0.1 * DISTANCE_UNIT_SS + 0.48))

    return model

def create_earth_system() -> CompoundModel:
    model: CompoundModel = CompoundModel()

    # Submodels
    model.add_model(create_earth())
    model.add_model(create_moon())

    # Movements
    model.add_movement(CyclicOrbit(365 * DAY_DURATION_SS, 1.2 * DISTANCE_UNIT_SS + 1.5))

    return model

def create_mars() -> Model:
    model: Model = Model("objects/mars/Mars 2K.obj")

    # Initial transformations
    model.add_starting_transformation(glm.scale(glm.vec3(0.1)))

    # Movements
    model.add_movement(CyclicRotation(40 * DAY_DURATION_SS))
    model.add_movement(CyclicOrbit(587 * DAY_DURATION_SS, 1.75 * DISTANCE_UNIT_SS + 1.5))

    return model

def create_solar_system() -> CompoundModel:
    model: CompoundModel = CompoundModel()

    # Submodels
    model.add_model(create_sun())
    model.add_model(create_mercury())
    model.add_model(create_venus())
    model.add_model(create_earth_system())
    model.add_model(create_mars())

    # Movements
    model.add_movement(ControledScale(2, 2))
    model.add_movement(ControledRotationBasedOnCamera(2))
    model.add_movement(ControledTranslationBasedOnCamera(15, glm.vec3(-5.0, 28.0, 40.0)))

    return model

models.append(create_solar_system())

##### Castle

In [572]:
def create_castle_floor() -> Model:
    model: Model = Model("objects/wood floor/Wood_Floor_001_OBJ.obj")

    # Initial transformations
    model.add_starting_transformation(glm.scale(glm.vec3(21.5,1.0,9.265)))

    # Final transformations
    model.add_ending_transformation(glm.translate(glm.vec3(0.0, 0.67, -20.59)))

    return model

def create_castle() -> Model:
    model: Model = Model("objects/castle/Castle OBJ.obj")

    # Final transformations
    model.add_ending_transformation(glm.translate(glm.vec3(0.0,-0.255,0.0)))

    return model

models.append(create_castle_floor())
models.append(create_castle())

##### Inner models

In [573]:
def create_cat() -> Model:
    model: Model = Model("objects/cat/12221_Cat_v1_l3.obj")

    # Initial transformations
    model.add_starting_transformation(glm.scale(glm.vec3(0.05)))
    model.add_starting_transformation(glm.rotate(3/2*glm.pi(), glm.vec3(1,0,0)))
    model.add_starting_transformation(glm.rotate(3/2*glm.pi(), glm.vec3(0,1,0)))

    # Final transformations
    model.add_ending_transformation(glm.translate(glm.vec3(15.0, 0.67, -14.0)))

    Illumination.add_light_source(model, glm.vec3(0.9,0.9,0.9), 35)

    return model

models.append(create_cat())

In [574]:
def create_dio() -> Model:
    model: Model = Model("objects/dio/Dio.obj")

    # Initial transformations
    model.add_starting_transformation(glm.scale(glm.vec3(0.02)))
    model.add_starting_transformation(glm.rotate(glm.pi()/4, glm.vec3(0,1,0)))

    # Movements
    model.add_movement(CyclicOrbit(25, 8, clockwise=False))

    # Final transformations
    model.add_ending_transformation(glm.translate(glm.vec3(-5.0, 0.67, -21.0)))

    return model

models.append(create_dio())

In [575]:
def create_chest() -> Model:
    model: Model = Model("objects/chest/chest_photoscan.obj")

    # Initial transformations
    model.add_starting_transformation(glm.scale(glm.vec3(4.0)))
    model.add_starting_transformation(glm.rotate(np.pi, glm.vec3(0,1,0)))

    # Final transformations
    model.add_ending_transformation(glm.translate(glm.vec3(10.0, 2.04, -30.0)))

    return model

models.append(create_chest())

##### Outside models

In [576]:
def create_well() -> Model:
    model: Model = Model("objects/well/Well.obj")

    # Initial transformations
    model.add_starting_transformation(glm.scale(glm.vec3(1.5)))
    model.add_starting_transformation(glm.rotate(3*glm.pi()/2, glm.vec3(0.0, 1.0, 0.0)))

    # Final transformations
    model.add_ending_transformation(glm.translate(glm.vec3(30.0, 2.2, 90.0)))

    return model

models.append(create_well())

In [577]:
def create_snowden() -> Model:
    model: Model = Model("objects/snowden/snow.obj")

    # Initial transformations
    model.add_starting_transformation(glm.scale(glm.vec3(2.5)))
    model.add_starting_transformation(glm.rotate(glm.pi()/2, glm.vec3(1,0,0)))

    # Final transformations
    model.add_ending_transformation(glm.translate(glm.vec3(0.0, 9.0, 130.0)))

    return model

models.append(create_snowden())

In [578]:
def create_trees() -> Model:
    model: Model = Model("objects/tree/arvorelowpoly.obj")

    # Initial transformations
    model.add_starting_transformation(glm.translate(glm.vec3(52.5, -1.0, 12.5)))
    model.add_starting_transformation(glm.scale(glm.vec3(3)))

    # Final transformations
    model.add_ending_transformation(glm.translate(glm.vec3(-10.0, 0.0, 50.0)))

    # Creates a model for rendering copies of the original model
    repeat_model: RepeatingModel = RepeatingModel(model)

    # Define the copies' offsets
    for i in range(1, 8):
        x = 0.0
        if i % 2 == 1:
            x = 20.0
        repeat_model.add_offset(glm.vec3(x, 0.0, i*10.0))

    return repeat_model

models.append(create_trees())

### Main Loop

In [579]:
class Timer:
    """
    A versatile timer class that tracks elapsed time, supports time acceleration,
    and calculates frames per second (FPS).

    Attributes:
        current_time (float): The current time in seconds, obtained using glfw.get_time().
        previous_time (float): The previous time in seconds, used for calculating delta time.
        delta_time (float): The elapsed time since the previous update, adjusted for time acceleration.\
            Defaults to 0.0.
        acceleration (float): The rate at which time is accelerated (positive) or decelerated (negative).\
            Defaults to 0.0.
        time_speed_up (float): A multiplier applied to delta_time for time acceleration. Defaults to 1.0 (normal speed).
    """
    current_time: float
    previous_time: float
    delta_time: float = 0.0

    acceleration: float = 0.0
    time_speed_up: float = 1.0

    @classmethod
    def setup(cls, time_to_double_speed: float | None = None) -> None:
        """
        Sets up the timer by initializing its internal variables.

        Args:
            time_to_double_speed (float, optional): The time it takes for the time scale to\
                                                    double (e.g., 1.0 for doubling speed in 1 second).\
                                                    If None, time scaling is disabled.
        """
        cls.current_time = glfw.get_time()
        cls.previous_time = cls.current_time
        cls.delta_time = 0.0

        cls.acceleration = 0.0
        if time_to_double_speed != None:
            cls.acceleration = 1/time_to_double_speed

    @classmethod
    def update_time(cls) -> None:
        """
        Updates the timer by calculating the delta time.

        This method updates the current time, calculates the delta time since the previous update,\
        and updates the previous time for the next frame. It also handles time scaling based on\
        user input (pressing M or N keys).
        """
        cls.current_time = glfw.get_time()
        cls.delta_time = cls.current_time - cls.previous_time
        cls.previous_time = cls.current_time

        # Change rate of time pass
        if KeyInputs.M == glfw.PRESS or KeyInputs.M == glfw.REPEAT:
            cls.time_speed_up *= glm.exp2(cls.acceleration * cls.delta_time)
        if KeyInputs.N == glfw.PRESS or KeyInputs.N == glfw.REPEAT:
            cls.time_speed_up *= glm.exp2(-cls.acceleration * cls.delta_time)

    @classmethod
    def get_delta_time(cls) -> float:
        """
        Returns the delta time scaled by the current time speed-up factor.
        """
        return cls.delta_time * cls.time_speed_up

    @classmethod
    @property
    def fps(cls) -> float:
        """
        Calculates and returns the current frames per second (FPS).
        """
        return 1/cls.delta_time

In [580]:
# Creates projection matrix
projection_matrix: glm.mat4 = Projection.get_projection_matrix()

def handle_window_resizing(window, width, height):
    WindowDim.update(width, height)

    # Updates the projection matrix
    global projection_matrix
    projection_matrix = Projection.get_projection_matrix()

    # Updates the window viewport
    glViewport(0, 0, width, height)

glfw.set_framebuffer_size_callback(window, handle_window_resizing)

In [581]:
# Background color (only visible in polygon mode)
bg_color: glm.vec3 = glm.vec3(0.25,0.25,0.25)
polygon_mode: bool = False

glfw.show_window(window)

Timer.setup(1)

while not glfw.window_should_close(window):
    # Reads the keyboard inputs
    glfw.poll_events()
    KeyInputs.get_inputs(window)

    # Closes the window
    if KeyInputs.ESC == glfw.PRESS:
        glfw.set_window_should_close(window, True)

    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(bg_color.x, bg_color.y, bg_color.z, 1.0)

    # Toggles polygon mode
    if KeyInputs.P == glfw.PRESS:
        polygon_mode = not polygon_mode

    if polygon_mode:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    else:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)

    # Updates the timer
    Timer.update_time()
    delta_time: float = Timer.get_delta_time()

    # Updates the camera position
    Camera.handle_movement(delta_time)

    loc = glGetUniformLocation(program, 'projection_matrix')
    glUniformMatrix4fv(loc, 1, GL_FALSE, projection_matrix.to_bytes())

    loc = glGetUniformLocation(program, 'view_matrix')
    glUniformMatrix4fv(loc, 1, GL_FALSE, Camera.get_view_matrix().to_bytes())

    #Computes the model transformations
    for model in models:
        model.handle_movement(delta_time)

    # Updates light position
    Illumination.update(delta_time)

    # Renders the models
    for model in models:
        model.render()

    glfw.swap_buffers(window)

glfw.terminate()

629.3662282079289
75.61722560399402
132.5521592746765
105.74401488876055
33.974892554401656
73.49050502675031
182.94914013903488
186.54976214908064
195.09911034804173
179.14404973039424
124.93284859387268
192.69679159844617
133.7005642163766
210.69931101324423
242.88351306715666
153.0104812179628
232.11550067310588
216.13677134894147
183.4357516279793
192.57804225161237
240.18253872944794
186.3411907202069
163.76260972094374
212.42697822626087
203.36363451484794
161.6527375891147
243.65284342870905
229.8956273851693
164.56571108843494
226.3775071308736
244.22410003418122
172.25045215745942
191.01831865675337
230.3988203580417
148.05602439962215
206.3685329261109
232.2125208991305
194.1219862561665
122.63919548687434
161.23051126195136
127.51361207808804
134.99649009126446
202.95090617578367
149.6938760235275
175.8272672926046
167.74301769689595
160.3977865105539
189.22191946714787
172.41676580629473
195.31250000000796
234.28531265376787
200.07202592932757
172.69367595758374
230.7337332